# TPS December 
i hope you'll find this notebook helpfull,
make sure to upvote it if you do.

i will be more than happy to hear any suggestions or answer any questions about this note book in comments :))
GOOD LUCK

In [ ]:
import gc
import seaborn as sns
import pandas as pd
import numpy as np
import sklearn 
import sklearn.utils
import matplotlib.pyplot as plt
import matplotlib
from sklearn.model_selection import *
from sklearn.preprocessing import *
from sklearn.metrics import *
from catboost import Pool, CatBoostClassifier 

import warnings 
warnings.filterwarnings('ignore')

In [ ]:
train_data = pd.read_csv("../input/tabular-playground-series-dec-2021/train.csv")
# test_data = pd.read_csv("../input/tabular-playground-series-dec-2021/test.csv")
# ids = pd.DataFrame(test_data['Id'].values, columns=["Id"])

In [ ]:
ids.head()

In [ ]:
train_data.head()

In [ ]:
train_data.describe()

In [ ]:
train_data.info()

In [ ]:
cols = train_data.columns.values.tolist()

In [ ]:
train_data.drop(train_data[train_data['Cover_Type']==5].index,inplace=True)
train_target = pd.DataFrame(train_data["Cover_Type"].values , columns=["Cover_Type"])
# test_data = test_data.drop(["Id"], axis=1)

In [ ]:
non_binary = ['Elevation','Aspect','Slope','Horizontal_Distance_To_Hydrology','Vertical_Distance_To_Hydrology','Horizontal_Distance_To_Roadways','Hillshade_9am','Hillshade_Noon','Hillshade_3pm','Horizontal_Distance_To_Fire_Points','Cover_Type']

# EDA

In [ ]:
t_corr = train_data[non_binary].corr()
plt.figure(figsize=(12,12))
sns.heatmap(t_corr, cmap='Blues', annot=True, annot_kws={'size':10})

In [ ]:
sns.catplot(x="Aspect", y="Cover_Type", kind="point", data=train_data, aspect=9, height=9)

In [ ]:
sns.relplot(x="Horizontal_Distance_To_Roadways", y="Horizontal_Distance_To_Fire_Points", hue="Cover_Type", data=train_data, aspect=5, height=5)

In [ ]:
# sns.catplot(x="Cover_Type",kind="count",data=train_data)

# Memmory Management

In [ ]:
non_binary = ['Elevation','Aspect','Slope','Horizontal_Distance_To_Hydrology','Vertical_Distance_To_Hydrology','Horizontal_Distance_To_Roadways','Hillshade_9am','Hillshade_Noon','Hillshade_3pm','Horizontal_Distance_To_Fire_Points']
binary = ['Wilderness_Area1','Wilderness_Area2','Wilderness_Area3','Wilderness_Area4','Soil_Type1','Soil_Type2','Soil_Type3','Soil_Type4','Soil_Type5','Soil_Type6','Soil_Type7','Soil_Type8','Soil_Type9','Soil_Type10','Soil_Type11','Soil_Type12','Soil_Type13','Soil_Type14','Soil_Type15','Soil_Type16','Soil_Type17','Soil_Type18','Soil_Type19','Soil_Type20','Soil_Type21','Soil_Type22','Soil_Type23','Soil_Type24','Soil_Type25','Soil_Type26','Soil_Type27','Soil_Type28','Soil_Type29','Soil_Type30','Soil_Type31','Soil_Type32','Soil_Type33','Soil_Type34','Soil_Type35','Soil_Type36','Soil_Type37','Soil_Type38','Soil_Type39','Soil_Type40']

In [ ]:
memory_usage = (train_data.memory_usage(deep=True) / 1024 ** 2) 
#+ (test_data.memory_usage(deep=True) / 1024 ** 2) 
start_mem = memory_usage.sum()

In [ ]:
for col in binary:
    train_data[col] = train_data[col].astype("int8")
#     test_data[col] = test_data[col].astype("int8")
for val in non_binary:
    train_data[val] = train_data[val].astype("float16")
#     test_data[val] = test_data[val].astype("float16")
    
memory_usage = (train_data.memory_usage(deep=True) / 1024 ** 2) 
# + (test_data.memory_usage(deep=True) / 1024 ** 2) 
end_mem = memory_usage.sum()
print("Mem. usage decreased from {:.2f} MB to {:.2f} MB ({:.2f}% reduction)".format(start_mem, end_mem, 100 * (start_mem - end_mem) / start_mem))
gc.collect()

In [ ]:
cols = train_data.columns.values.tolist()
i = 1
plt.figure()
fig, ax = plt.subplots(8, 3,figsize=(32, 38))
for feature in cols[1:25]:
    plt.subplot(8, 3,i)
    sns.histplot(train_data[feature],color="green", kde=True,bins=100, label=feature)
    plt.xlabel(feature, fontsize=9); plt.legend()
    i += 1
plt.show()

In [ ]:
cols = train_data.columns.values.tolist()
i = 1
plt.figure()
fig, ax = plt.subplots(10, 3,figsize=(32, 38))
for feature in cols[25:55]:
    plt.subplot(10, 3,i)
    sns.histplot(train_data[feature],color="purple", kde=True,bins=100, label=feature)
    plt.xlabel(feature, fontsize=9); plt.legend()
    i += 1
plt.show()

In [ ]:
train_data.isnull().sum()

In [ ]:
train_data = train_data.drop(["Id","Cover_Type"], axis=1)    

In [ ]:
test_data.isnull().sum()

# Data Prep

In [ ]:
MX = MinMaxScaler()
train_data = pd.DataFrame(MX.fit_transform(train_data.values), columns=[train_data.columns.tolist()])
test_data = pd.DataFrame(MX.fit_transform(test_data.values), columns=[test_data.columns.tolist()])

In [ ]:
# Adding some new features to make the final model more accurate ...
train_data["mean"]=train_data.mean(axis=1)
train_data["min"]=train_data.min(axis=1)
train_data["var"]=train_data.var(axis=1)
train_data["std"]=train_data.std(axis=1)
#///////////////////////////////////////
test_data["mean"]=test_data.mean(axis=1)
test_data["min"]=test_data.min(axis=1)
test_data["var"]=test_data.var(axis=1)
test_data["std"]=test_data.std(axis=1)

In [ ]:
test_data.head()

In [ ]:
train_data.tail()

In [ ]:
# Splitting Data
X_train, X_test, y_train, y_test = train_test_split(train_data, train_target, test_size=0.2, random_state=1)

In [ ]:
test_model = CatBoostClassifier(random_state=0 ,task_type="GPU").fit(X_train, y_train)

In [ ]:
test_model.score(X_test, y_test)

In [ ]:
parameters = [{
                         'learning_rate':[0.2],                   
                         'verbose':[True],
                         'eval_metric':['Accuracy'],
                         'subsample':[0.8],
                         'max_depth':np.arange(5,30,5),
                         'n_estimators':[2000],
                         'colsample_bylevel':np.arange(0.1,0.9,0.1),
                         'random_state':[2],
                         'reg_lambda':np.arange(0.1,10,0.1),
                         'min_child_samples':np.arange(5,50,5),
                         'num_leaves':np.arange(1,20,1),
                         'task_type':['GPU']
}]

In [ ]:
grid_search_model = GridSearchCV(estimator = test_model,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 5,
                           n_jobs = -1)

In [ ]:
grid_search_model.fit(X_train, y_train)

In [ ]:
best_accuracy = grid_search_model.best_score_
print("Best Accuracy: {:.2f} %".format(best_accuracy*100))

In [ ]:
best_parameters = grid_search_model.best_params_
print("Best Parameters:", best_parameters)

In [ ]:
# parameters = [{
#                          'learning_rate':[],                   
#                          'verbose':[True],
#                          'eval_metric':['Accuracy'],
#                          'subsample':[0.8],
#                          'max_depth':'',
#                          'n_estimators':[],
#                          'colsample_bylevel':'',
#                          'random_state':'',
#                          'reg_lambda':'',
#                          'min_child_samples':'',
#                          'num_leaves':'',
#                          'task_type':['GPU']
# }]

In [ ]:
model_2 = CatBoostClassifier(**params).fit(X_train, y_train)

In [ ]:
model_2.score(X_test, y_test)
# First  Model 0.95994
# Second Model 0.96109
# Third  Model 0.96081
# Fourth Model 0.95993
# Fifth  Model 0.960715

In [ ]:
prediction = model_2.predict(test_data)

In [ ]:
preds = pd.DataFrame(prediction, columns = ["Cover_Type"])

In [ ]:
preds["Cover_Type"].value_counts(normalize = True)*100

In [ ]:
sns.catplot(x="Cover_Type",kind='count',data=preds,palette="ch:.95")

In [ ]:
result = pd.concat([ids,preds],axis=1)

In [ ]:
result.head()

In [ ]:
result.to_csv("submission.csv",index=False)

In [ ]:
# M.Hossein Hashemi